# TESTING OUT STUFF IN THIS FILE, CAN BE DELETED WHEN WE ARE DONE.

In [1]:
import os,sys,inspect
ConvCRF_dir = '../ConvCRF'
sys.path.insert(0,ConvCRF_dir)
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, utils, models
import torchvision.transforms.functional as TF

import numpy as np
import glob
import matplotlib.pyplot as plt
import random
import os
from torch.utils.tensorboard import SummaryWriter

from PIL import Image
from torchsummary import summary

import cv2

In [3]:
from tools.dataloader import DatasetFloe, DatasetFloeVal  # Init signature: DatasetFloe(patch_size: int, batch_size: int, mode='train')
from tools.loss import calc_loss          # smoothening factor for dice loss is 1e-6
from tools.crf_conf import default_conf   # configuration for conditional random field. 

In [4]:
# def model_save(model, total_iters, best_model):
#     if best_model == 'best_model':
#         path = '../model/weights/'+ps+'_'+arc+'_' + loss+'_BEST_.mdl'
#     else:
#         path = '../model/weights/'+ps+'_'+arc+'_' + str(total_iters) + 'iter'+'_'+loss+'.mdl'
    
#     torch.save({'state_dict':model.state_dict(),
#             'iterations':str(total_iters),
#             'weights':loss,
#             'patch_size':ps
#            }, path)

In [4]:
def model_train(model1, train_loader, learning_rate, weight_bce):
    model1.cuda()
    model1.train()
    
    total_iters = 0
    
    for i in range(n_iterations):
        for batch_idx, sample in enumerate(train_loader):
            
            model1.train()
            
            X = sample['image'].cuda()
            y = sample['mask'].cuda()
            
            optimizer.zero_grad()
            
            pred1 = model1(X)    # USE pred['out'] for dlv3 and fcn
            
            if (not(str(type(pred1)) == "<class 'collections.OrderedDict'>")):
                loss = calc_loss(pred1, y[:,1,:,:].unsqueeze(1), weight_bce=weight_bce)
            else:
                loss = calc_loss(pred1['out'], y[:,1,:,:].unsqueeze(1), weight_bce=weight_bce)
            
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            total_iters +=1
            
            print("Iteration: {}\tCurrent loss:{:.6f}".format(total_iters, loss.item()))
            
            writer.add_scalar('training', loss.item(), total_iters)
            writer.add_image('mask', y[:,1,:,:].unsqueeze(1), total_iters, dataformats='NCHW')
            if (not(str(type(pred1)) == "<class 'collections.OrderedDict'>")):
                writer.add_image('prediction', torch.sigmoid(pred1)[:,0,:,:].unsqueeze(1), total_iters, dataformats='NCHW') # torch.sigmoid(pred1)[:,1,:,:] resunet
            else:
                writer.add_image('prediction', torch.sigmoid(pred1['out'])[:,0,:,:].unsqueeze(1), total_iters, dataformats='NCHW')
            
            if(total_iters % 20 == 0):
                model1.eval()
                val_loss_total = []
                with torch.no_grad():
                    for _, sample_val in enumerate(val_loader):
                        
                        X_val = sample_val['image'].cuda()
                        y_val = sample_val['mask'].cuda()
                        pred1_val = model1(X_val)    # USE pred['out'] for dlv3 and fcn
            
                        if (not(str(type(pred1)) == "<class 'collections.OrderedDict'>")):
                            loss_val = calc_loss(pred1_val, y_val[:,1,:,:].unsqueeze(1), weight_bce=weight_bce)
                        else:
                            loss_val = calc_loss(pred1_val['out'], y_val[:,1,:,:].unsqueeze(1), weight_bce=weight_bce)
                        
                        val_loss_total.append(loss_val.item())
                        
                writer.add_scalar('validation', np.mean(val_loss_total), total_iters)
                print("Validation loss:{:.6f}".format(np.mean(val_loss_total)))
                
            writer.close()

In [6]:
# def train_model(model, train_loader, learning_rate, weight_bce):
#     model.cuda()
#     model.train()
#     total_iters = 0
#     best_loss = 999999
    
#     for i in range(n_iterations):
#         for batch_idx, sample in enumerate(train_loader):
#             X = sample['image'].cuda()
#             y = sample['mask'].cuda()
            
#             optimizer.zero_grad()
            
#             pred = model(X)    # USE pred['out'] for dlv3 and fcn,,,,,,,, ELSE USE pred    :)
#             loss = calc_loss(pred, y, weight_bce=weight_bce)
# #             out = torch.sigmoid(pred)
# #             loss = F.mse_loss(pred, y)
# #             bce = F.binary_cross_entropy_with_logits(pred, y)
# #             lovasz_loss = lovasz_softmax(out, y, classes=[1])
# #             loss = weight_bce * bce + (1-weight_bce) * lovasz_loss
    
            
#             loss.backward()
#             optimizer.step()
#             scheduler.step()
            
#             total_iters +=1
            
#             print("Iteration: {}\tCurrent loss:{:.6f}".format(total_iters,loss.item()))
            
#             writer.add_scalar('training', loss.item(), total_iters)
            
#             if (total_iters % validation_counter == 0):
#                 valid_loader =  DataLoader(valid_ds, batch_size=6, num_workers=0)
#                 validate(model, valid_loader, total_iters, weight_bce)
#             writer.close()
            
#             model.train()
            
# #             if(total_iters>1 and loss.item() < best_loss):
# #                 best_loss = loss.item()
# #                 model_save(model, total_iters, 'best_model')
# #                 print('SAVED---------------------------->')
            
#         #if(total_iters % 200 == 0 and total_iters < 1000 or total_iters % 1000 == 0):
#             #model_save(model, total_iters, 'regular_saving')

In [16]:
# # """DEEPLAB V3 RESNET101"""

# from torchvision.models.segmentation.deeplabv3 import DeepLabHead
# model1 = models.segmentation.deeplabv3_resnet101(pretrained=True)
# model1.classifier = DeepLabHead(2048, 1)

In [17]:
# model.load_state_dict(torch.load('../model/weights/2_384_DLv3_200iter_MSE.mdl')['state_dict'])

In [18]:
# # """FCN RESNET101"""

# from torchvision.models.segmentation.fcn import FCNHead
# model = models.segmentation.fcn_resnet101(pretrained=True)
# model.classifier = FCNHead(2048, 1)

In [19]:
# model.load_state_dict(torch.load('../model/weights/1_288_FCN_200iter_0.5BCE.mdl')['state_dict'])

In [20]:
# """RESUNET BACKBONE RESNET34"""
# from model.architecture import ResUNet
# resnet = models.resnet34(pretrained=True)
# layers = list(resnet.children())
# model = ResUNet(layers, 2)

In [5]:
from backboned_unet import Unet
# 'resnet'- 18, 34, 50, 101, 152
# 'vgg'- 16, 19
# 'densenet' - 121, 161, 169, 201

model = Unet(backbone_name='densenet121', classes=1)

upsample_blocks[0] in: 1024   out: 256
upsample_blocks[1] in: 256   out: 128
upsample_blocks[2] in: 128   out: 64
upsample_blocks[3] in: 64   out: 32
upsample_blocks[4] in: 32   out: 16


In [22]:
# model.load_state_dict(torch.load('../model/weights/resUNET continuous training/288_resUNET_200iter_0.5BCE.mdl')['state_dict'])

In [6]:
ps = '480'
arc = 'denseUNET' #'DLv3'
loss = '0.5BCE'

In [7]:
batch_size = 8

train_ds = DatasetFloe(patch_size= int(ps), batch_size= batch_size, mode='train')
train_loader =  DataLoader(train_ds, batch_size=batch_size, num_workers=0)

val_ds = DatasetFloeVal(patch_size= int(ps),)  # ADDED LATER
val_loader =  DataLoader(val_ds, batch_size=1, num_workers=0)  # ADDED LATER

In [12]:
n_iterations = 50
learning_rate = 10e-2
weight_bce = 1

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), 
    lr= learning_rate)#,
#         weight_decay = weight_decay)

scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer,
    milestones=[2000],
    gamma = 0.1
)

writer = SummaryWriter()

In [13]:
model = torch.load('/home/muzik999/MASc/iceFloe/floes/model/weights/UNET_CRF/NEW/experiment_BACKBONE/1_480_DENSEUNET_.mdl')

In [14]:
model_train(model, train_loader, learning_rate, weight_bce)

Iteration: 1	Current loss:0.046977
Iteration: 2	Current loss:0.031361
Iteration: 3	Current loss:0.037066
Iteration: 4	Current loss:0.047208
Iteration: 5	Current loss:0.028644
Iteration: 6	Current loss:0.014107
Iteration: 7	Current loss:0.024451
Iteration: 8	Current loss:0.039960
Iteration: 9	Current loss:0.023345
Iteration: 10	Current loss:0.023380
Iteration: 11	Current loss:0.021914
Iteration: 12	Current loss:0.032365
Iteration: 13	Current loss:0.044724
Iteration: 14	Current loss:0.023831
Iteration: 15	Current loss:0.013599
Iteration: 16	Current loss:0.046493
Iteration: 17	Current loss:0.022152


KeyboardInterrupt: 

In [15]:
torch.save(model, '/home/muzik999/MASc/iceFloe/floes/model/weights/UNET_CRF/NEW/2_480_DENSEUNET_.mdl')

In [8]:
model1.cuda()
model1.train()

total_iters = 0

for i in range(n_iterations):
    for batch_idx, sample in enumerate(train_loader):

        model1.train()

        X = sample['image'].cuda()
        y = sample['mask'].cuda()

        optimizer.zero_grad()

        pred1 = model1(X)    # USE pred['out'] for dlv3 and fcn

        if (not(str(type(pred1)) == "<class 'collections.OrderedDict'>")):
            loss = calc_loss(pred1, y, weight_bce=weight_bce)
        else:
            loss = calc_loss(pred1['out'], y[:,1,:,:].unsqueeze(1), weight_bce=weight_bce)

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_iters +=1

        print("Iteration: {}\tCurrent loss:{:.6f}".format(total_iters, loss.item()))

        writer.add_scalar('training', loss.item(), total_iters)
        writer.add_image('mask', y[:,1,:,:].unsqueeze(1), total_iters, dataformats='NCHW')
        if (pred1.shape[1] == 2):
            writer.add_image('prediction', torch.sigmoid(pred1)[:,1,:,:].unsqueeze(1), total_iters, dataformats='NCHW')
        else:
            writer.add_image('prediction', torch.sigmoid(pred1['out'])[:,0,:,:].unsqueeze(1), total_iters, dataformats='NCHW')

        if(total_iters % 20 == 0):
            model1.eval()
            val_loss_total = []
            with torch.no_grad():
                for _, sample_val in enumerate(val_loader):

                    X_val = sample_val['image'].cuda()
                    y_val = sample_val['mask'].cuda()
                    pred1_val = model1(X_val)    # USE pred['out'] for dlv3 and fcn

                    if (not(str(type(pred1)) == "<class 'collections.OrderedDict'>")):
                        loss_val = calc_loss(pred1_val, y_val, weight_bce=weight_bce)
                    else:
                        loss_val = calc_loss(pred1['out'], y_val[:,1,:,:].unsqueeze(1), weight_bce=weight_bce)

                    val_loss_total.append(loss_val.item())

            writer.add_scalar('validation', np.mean(val_loss_total), total_iters)
            print("Validation loss:{:.6f}".format(np.mean(val_loss_total)))

        writer.close()

ValueError: Target size (torch.Size([4, 1, 480, 480])) must be the same as input size (torch.Size([4, 2, 480, 480]))

In [44]:
class Inception1(nn.Module):

    def __init__(self, in_channels, pool_features):
        super(Inception1, self).__init__()
        self.branch1x1 = conv(in_channels, pool_features, kernel_size=1) # 1

        self.branch5x5_1 = conv(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = conv(48, pool_features, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = conv(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = conv(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = conv(96, pool_features, kernel_size=3, padding=1)

        self.branch_pool = conv(in_channels, pool_features, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

    
class Inception2(nn.Module):

    def __init__(self, in_channels, pool_features):
        super(Inception2, self).__init__()
        self.branch1x1 = conv(in_channels, pool_features, kernel_size=1) # 1

        self.branch5x5_1 = conv(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = conv(48, pool_features, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = conv(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = conv(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = conv(96, pool_features, kernel_size=3, padding=1)

        self.branch_pool = conv(in_channels, pool_features, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

class Inception3(nn.Module):

    def __init__(self, in_channels, pool_features):
        super(Inception3, self).__init__()
        self.branch1x1 = conv(in_channels, pool_features, kernel_size=1) # 1

        self.branch5x5_1 = conv(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = conv(48, pool_features, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = conv(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = conv(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = conv(96, pool_features, kernel_size=3, padding=1)

        self.branch_pool = conv(in_channels, pool_features, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

class Inception4(nn.Module):

    def __init__(self, in_channels, pool_features):
        super(Inception4, self).__init__()
        self.branch1x1 = conv(in_channels, pool_features, kernel_size=1) # 1

        self.branch5x5_1 = conv(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = conv(48, pool_features, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = conv(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = conv(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = conv(96, pool_features, kernel_size=3, padding=1)

        self.branch_pool = conv(in_channels, pool_features, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)


class incepUnet(nn.Module):
    def __init__(self, n_classes):
        super().__init__()

        self.dconv_d1 = Inception1(1, 16)
        self.dconv_d2 = Inception2(64, 32)
        self.dconv_d3 = Inception3(128, 64)
        self.dconv_d4 = Inception4(256, 128)

        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_u3 = double_conv(512 + 256, 256)  # skip connection, so adding 256
        self.dconv_u2 = double_conv(256 + 128, 128)
        self.dconv_u1 = double_conv(128 + 64, 64)

        self.conv_last = nn.Conv2d(64, n_classes, 1, dilation=1)

    def forward(self, x):
        conv1 = self.dconv_d1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_d2(x)
        x = self.maxpool(conv2)

        conv3 = self.dconv_d3(x)
        x = self.maxpool(conv3)

        x = self.dconv_d4(x)

        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)

        x = self.dconv_u3(x)

        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)

        x = self.dconv_u2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)

        x = self.dconv_u1(x)

        out = self.conv_last(x)

        return out

In [57]:
def conv(in_chnls, out_chnls, kernel_size, padding=0):
    return nn.Sequential(
        nn.Conv2d(in_channels = in_chnls, out_channels = out_chnls, kernel_size = kernel_size, padding = padding),
        nn.BatchNorm2d(num_features = out_chnls),
        nn.ReLU(inplace = True)
    )
def double_conv(in_channels, out_channels):

    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace = True),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace = True)
    )

In [58]:
incepUnet(1)

incepUnet(
  (dconv_d1): Inception1(
    (branch1x1): Sequential(
      (0): Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (branch5x5_1): Sequential(
      (0): Conv2d(1, 48, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (branch5x5_2): Sequential(
      (0): Conv2d(48, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (branch3x3dbl_1): Sequential(
      (0): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (branch3x3dbl_2): Sequential(
      (0): Conv2d(64, 96, kernel_size=(

In [59]:
summary(incepUnet(1).cuda(), (1,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]              32
       BatchNorm2d-2         [-1, 16, 224, 224]              32
              ReLU-3         [-1, 16, 224, 224]               0
            Conv2d-4         [-1, 48, 224, 224]              96
       BatchNorm2d-5         [-1, 48, 224, 224]              96
              ReLU-6         [-1, 48, 224, 224]               0
            Conv2d-7         [-1, 16, 224, 224]          19,216
       BatchNorm2d-8         [-1, 16, 224, 224]              32
              ReLU-9         [-1, 16, 224, 224]               0
           Conv2d-10         [-1, 64, 224, 224]             128
      BatchNorm2d-11         [-1, 64, 224, 224]             128
             ReLU-12         [-1, 64, 224, 224]               0
           Conv2d-13         [-1, 96, 224, 224]          55,392
      BatchNorm2d-14         [-1, 96, 2

In [60]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /home/muzik999/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /home/muzik999/.cache/torch/checkpoints/inception_v3_google-1a9a5a14.pth
100%|██████████| 104M/104M [00:10<00:00, 10.4MB/s] 


In [64]:
list_layers = list(model.children())

In [ ]:
summary()

In [65]:
len(list_layers)

18

In [66]:
list_layers

[BasicConv2d(
   (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
   (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ), BasicConv2d(
   (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ), BasicConv2d(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ), BasicConv2d(
   (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ), BasicConv2d(
   (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
   (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
 ), InceptionA(
   (branch1x1): BasicConv2d(
     (conv): Conv2d(192, 64, kernel_size=(1, 1